In [2]:
# Auto reload so I don't need to restart the kernel each time
%load_ext autoreload
%autoreload 2

In [3]:
# Generate ID3 decision tree
# Required: Don't use scikit
import numpy as np
import pandas as pd
import json
import hashlib
import networkx as nx
from treelib import Node, Tree
from contextlib import redirect_stdout
from networkx.drawing.nx_pydot import graphviz_layout
#from code.plotting import plotting 

In [5]:
## Import data
FILE = '../data/put-titanic-homework.csv'
columns_to_omit = [
    'Name']  # Omit name for simplicity, we assume it doesn't affect the chances of survival (it musn't be the case! Our assumption)
input_csv_dataframe = pd.read_csv(FILE, usecols=lambda column: column not in columns_to_omit)
## PREPROCESS
# Map age to one of 3 categories
# Age[0,20] = young || Age[20,40] = middle || Age[40,99999] = old
input_csv_dataframe['Age'] = input_csv_dataframe['Age'].map(
    lambda x: 'young' if 0 <= x <= 20 else ('middle' if 20 < x <= 40 else 'old'))
print(f"input_csv_dataframe: \n {input_csv_dataframe.head(n=5)}")

input_csv_dataframe: 
    PassengerId  Pclass     Sex     Age  SibSp  Parch  Survived
0            1       3    male  middle      1      0         0
1            2       1  female  middle      1      0         1
2            3       3  female  middle      0      0         1
3            4       1  female  middle      1      0         1
4            5       3    male  middle      0      0         0


In [6]:
# Calculating the entropy of the whole dataset
def calc_value_entropy(feature, value_key, df):
    survived_sum_by_value = df.loc[df[feature] == value_key, 'Survived'].sum()
    p_survived = survived_sum_by_value / df.shape[0]
    p_no_survived = 1 - p_survived
    entropy = (- p_no_survived * np.log2(p_no_survived, where=(p_no_survived != 0)) -
               p_survived * np.log2(p_survived, where=(p_survived != 0)))
    return entropy


whole_dataset_entropy = calc_value_entropy('Survived', 1, input_csv_dataframe)
print(f"Complete whole_dataset_entropy: {whole_dataset_entropy}")

Complete whole_dataset_entropy: 0.9709505944546686


In [7]:
# Calculating the entropy for the filtered dataset
features_data_list = []
for column in input_csv_dataframe.columns:

    if column == 'Survived' or column == 'PassengerId':
        continue  # because these are not features

    # Get all unique values from column
    values_set = set(input_csv_dataframe[column].unique())

    for value in values_set:
        features_dict = {}
        features_dict['Feature'] = column
        features_dict['Value'] = value
        features_dict['P'] = input_csv_dataframe[column].eq(value).sum() / len(input_csv_dataframe)
        features_dict['Cases'] = input_csv_dataframe.loc[input_csv_dataframe[column] == value, 'PassengerId'].to_list()
        features_dict['Entropy'] = calc_value_entropy(column, value, input_csv_dataframe)
        features_data_list.append(features_dict)

print(*features_data_list, sep='\n')

{'Feature': 'Pclass', 'Value': 1, 'P': 0.21, 'Cases': [2, 4, 7, 12, 24, 28, 31, 32, 35, 36, 53, 55, 56, 62, 63, 65, 84, 89, 93, 97, 98], 'Entropy': 0.5293608652873644}
{'Feature': 'Pclass', 'Value': 2, 'P': 0.19, 'Cases': [10, 16, 18, 21, 22, 34, 42, 44, 54, 57, 59, 67, 71, 73, 79, 82, 85, 99, 100], 'Entropy': 0.5293608652873644}
{'Feature': 'Pclass', 'Value': 3, 'P': 0.6, 'Cases': [1, 3, 5, 6, 8, 9, 11, 13, 14, 15, 17, 19, 20, 23, 25, 26, 27, 29, 30, 33, 37, 38, 39, 40, 41, 43, 45, 46, 47, 48, 49, 50, 51, 52, 58, 60, 61, 64, 66, 68, 69, 70, 72, 74, 75, 76, 77, 78, 80, 81, 83, 86, 87, 88, 90, 91, 92, 94, 95, 96], 'Entropy': 0.6343095546405662}
{'Feature': 'Sex', 'Value': 'male', 'P': 0.6, 'Cases': [1, 5, 6, 7, 8, 13, 14, 17, 18, 21, 22, 24, 27, 28, 30, 31, 34, 35, 36, 37, 38, 43, 46, 47, 49, 51, 52, 55, 56, 58, 60, 61, 63, 64, 65, 66, 68, 70, 71, 73, 74, 75, 76, 77, 78, 79, 81, 82, 87, 88, 90, 91, 92, 93, 94, 95, 96, 97, 98, 100], 'Entropy': 0.36592365090022333}
{'Feature': 'Sex', 'Val

In [10]:
# Save calculated [ Feature, Value, P, Cases, Entropy ] to "features_entropy.csv"
features_entropy_df = pd.DataFrame(features_data_list)
features_entropy_df.to_csv("../data/calculated_features_entropy.csv", sep=',', index=False, encoding='utf-8')
print(f"features_data_dataframe: \n {features_entropy_df.head(n=4)}")

features_data_dataframe: 
   Feature Value     P                                              Cases  \
0  Pclass     1  0.21  [2, 4, 7, 12, 24, 28, 31, 32, 35, 36, 53, 55, ...   
1  Pclass     2  0.19  [10, 16, 18, 21, 22, 34, 42, 44, 54, 57, 59, 6...   
2  Pclass     3  0.60  [1, 3, 5, 6, 8, 9, 11, 13, 14, 15, 17, 19, 20,...   
3     Sex  male  0.60  [1, 5, 6, 7, 8, 13, 14, 17, 18, 21, 22, 24, 27...   

    Entropy  
0  0.529361  
1  0.529361  
2  0.634310  
3  0.365924  


In [9]:
# Calculate information gain and information split gain
def calc_conditional_entropy(feature, df):
    return (df.loc[df['Feature'] == feature, 'P'] * df.loc[df['Feature'] == feature, 'Entropy']).sum()


def calc_split_gain(feature, df):
    split_gain = 0
    unique_values = set(df[column].unique())
    for value in unique_values:
        p = df[(df[feature] == value)].shape[0] / df.shape[0]
        split_gain -= p * np.log2(p, where=(p != 0))
    return split_gain


features_by_information_gains = []
for df_column in input_csv_dataframe.columns:

    if df_column == 'Survived' or df_column == 'PassengerId':
        continue  # because these are not features

    info_gain = whole_dataset_entropy - calc_conditional_entropy(df_column, features_entropy_df)
    info_split_gain = calc_split_gain(df_column, input_csv_dataframe)
    features_entropy_df.loc[features_entropy_df["Feature"] == df_column, "InfoGain"] = info_gain
    features_entropy_df.loc[features_entropy_df["Feature"] == df_column, "InfoSplit"] = info_split_gain

feat_df_sort_by_gain = features_entropy_df.sort_values(by='InfoGain', ascending=False)
print(feat_df_sort_by_gain.head(n=4))

  Feature   Value     P                                              Cases  \
3     Sex    male  0.60  [1, 5, 6, 7, 8, 13, 14, 17, 18, 21, 22, 24, 27...   
4     Sex  female  0.40  [2, 3, 4, 9, 10, 11, 12, 15, 16, 19, 20, 23, 2...   
0  Pclass       1  0.21  [2, 4, 7, 12, 24, 28, 31, 32, 35, 36, 53, 55, ...   
2  Pclass       3  0.60  [1, 3, 5, 6, 8, 9, 11, 13, 14, 15, 17, 19, 20,...   

    Entropy  InfoGain  InfoSplit  
3  0.365924  0.385426   0.000000  
4  0.914926  0.385426   0.000000  
0  0.529361  0.378621   0.472823  
2  0.634310  0.378621   0.472823  


In [89]:
# Build tree based on features_entropy_df
def ID3(df, tree):
    if tree.size() == 0:  # if tree empty
        max_gain_feature_index = df["InfoGain"].idxmax()
        root = tree.create_node(df.loc[max_gain_feature_index, "Feature"])
        
        return
    else:
        best_feature = df.loc[df['InfoGain'].idxmax()]
    #     root = Node(best_feature['Feature'])
    #     for value in best_feature['Value']:
    #         df_v = df.loc[df[best_feature['Feature']] == value]
    #         child = ID3(df_v)
    #         root.add_child(value, child)
    #     return root


tree = Tree()
print(ID3(feat_df_sort_by_gain, tree))
tree.show()


# def ID3(D, A):
#     if D.empty or A.empty:
#         return D['Survived'].mode().iloc[0]
#     else:
#         A_best = A.loc[A['InfoGain'].idxmax()]
#         root = Node(A_best['Feature'])
#         for v in A_best['Value']:
#             D_v = D.loc[D[A_best['Feature']] == v]
#             child = ID3(D_v, A.drop(A_best))
#             root.add_child(v, child)
#         return root
# 
# 
# def ID3(D, A):
#   wenn D rein oder A leer ist:
#     Gibt einen Blattknoten mit der Mehrheitsklasse in D zurück
#   anders:
#     A_best = argmax(InformationGain(D, A))
#     root = Node(A_best)
#     für v in Werten (A_best):
#       D_v = Teilmenge(D, A_best, v)
#       child = ID3(D_v, A - {A_best})
#       root.add_child(v, child)
#     Rückkehr zur Wurzel


None
b'Sex\n'


In [6]:
# Build tree basing on features_entropy_df


## VISUALIZE
# Create json
tree = Tree()
tree.create_node(features_by_information_gains[0][0], 0)
for index, feature in enumerate(features_by_information_gains):
    if index == 0:
        continue
    feature_df = features_entropy_df.loc[["Feature"] == feature[0]]
    values = features_entropy_df.loc[features_entropy_df["Feature"] == feature[0], 'Value'].to_list()
    for value in values:
        hashy = int(hashlib.sha256(str(value).encode('utf-8')).hexdigest(), 16)

        # if all examples were survivors make "survived" node
        # if all dies make "died" node
        # if neither 
        tree.create_node(features_by_information_gains[index][0], hashy, parent=index - 1, data=value)

with open('tree.json', 'w') as f:
    print(tree.to_json(with_data=True), file=f)

# Print decision tree using generated json
# 4) Draw decision tree (use package) with info how many
data = json.loads(tree.to_json())
edges = []


def get_edges(treedict, parent=None):
    name = next(iter(treedict.keys()))
    if parent is not None:
        edges.append((parent, name))
    for item in treedict[name]["children"]:
        if isinstance(item, dict):
            get_edges(item, parent=name)
        else:
            edges.append((name, item))


get_edges(data)

# Dump edge list in Graphviz DOT format
with open('tree.dot', 'w') as f:
    print('strict digraph tree {', file=f)
    for row in edges:
        print('    {0} -> {1} [ label="value" ];'.format(*row), file=f)
    print('}', file=f)


# G = nx.DiGraph()
# for index, feature in enumerate(features_by_gains):
#     G.add_node(features_by_gains[index][0])
#     feature_possible_choices = set(features_entropy_df.loc[features_entropy_df["Feature"] == features_by_gains[index][0], "Value"])
#     G.add_nodes_from(feature_possible_choices)
#     for choice in feature_possible_choices:
#         G.add_edge(features_by_gains[index][0], choice)
# 
# pos = graphviz_layout(<G, prog="dot")
# nx.draw(G, pos, with_labels=True, font_weight='bold')

KeyError: 'False: boolean label can not be used without a boolean index'

In [ ]:
start by writing function 